In [33]:
import folium
import pandas as pd 
import numpy as np

from bokeh.palettes import plasma 

import branca
import branca.colormap as cm

In [49]:
df = pd.read_csv("../Data/AB_data_clean.csv").dropna(subset=['latitude', 'longitude'])
df_trees = pd.read_csv("../Data/new_york_tree_census_2015.csv").dropna(subset=['latitude', 'longitude'])
df_rats = pd.read_csv("../Data/Rat_Sightings.csv").dropna(subset=['Latitude', 'Longitude'])
#df_places = pd.read_excel("../Data/data_places/New_York_Tourist_Locations.xlsx").dropna(subset=['latitude', 'longitude'])

In [50]:
print('Datapoints in Air BnB data:', df.shape[0])
print('Datapoints in Tree data:', df_trees.shape[0])
print('Datapoints in Rat data:', df_rats.shape[0])
#print('Datapoints in Touristic data:', df_places.shape[0])

Datapoints in Air BnB data: 48881
Datapoints in Tree data: 683788
Datapoints in Rat data: 101208


In [51]:
# Making sure the relative frequency of the three data types are preserved in the plot
data_price = df.sample(round(df.shape[0]/100))
data_tree = df_trees.sample(round(df_trees.shape[0]/1000))
data_rats = df_rats.sample(round(df_rats.shape[0]/1000))
#data_places = df_places.sample(round(df_places.shape[0]/1))

print('Datapoints in Air BnB plot data:', data_price.shape[0])
print('Datapoints in Tree plot data:', data_tree.shape[0])
print('Datapoints in Rat plot data:', data_rats.shape[0])
#print('Datapoints in Touristic Places plot data:', data_places.shape[0])

Datapoints in Air BnB plot data: 489
Datapoints in Tree plot data: 684
Datapoints in Rat plot data: 101


## Version 1

In [52]:
#getting colors to match price
color_map = plasma(20)
lin_space = np.linspace(data_price.price.min(), data_price.price.max(), 21)
col_idx_func = lambda x, lin: np.argwhere(x <= lin)[0][0]-1

In [53]:
#creating to different marker groups, one for each time period
BnB_prices = folium.FeatureGroup(name='<span style=\\"color: #0C0786;\\">Air BnBs (color after price) </span>')
Trees = folium.FeatureGroup(name='<span style=\\"color: #79D151;\\">Trees</span>')
Rats = folium.FeatureGroup(name='<span style=\\"color: #C0C0C0;\\">Rat Sightings</span>')
#Places = folium.FeatureGroup(name='<span style=\\"color: #5AD8CD;\\">Touristic Places</span>')

In [54]:
def add_marker_price(x):
      col_idx = col_idx_func(x.price, lin_space)
      folium.CircleMarker(location=[x.latitude, x.longitude], radius= 2, color=color_map[col_idx], weight = 2,  fill_color=color_map[col_idx], fill=True, fill_opacity=1).add_to(BnB_prices)

def add_marker_tree(x):
      folium.CircleMarker(location=[x.latitude, x.longitude], radius= 1, color='#79D151', weight = 2,  fill_color='#79D151', fill=True, fill_opacity=1).add_to(Trees)

def add_marker_rat(x):
      folium.CircleMarker(location=[x.Latitude, x.Longitude], radius= 1, color='#C0C0C0', weight = 2,  fill_color='#C0C0C0', fill=True, fill_opacity=1).add_to(Rats)

def add_marker_place(x):
      folium.CircleMarker(location=[x.latitude, x.longitude], radius= 1, color='#5AD8CD', weight = 2,  fill_color='#5AD8CD', fill=True, fill_opacity=1).add_to(Places)

In [56]:
NYC_coor = (40.730610, -73.935242)

NY_map = folium.Map(NYC_coor, zoom_start=11, tiles = 'cartodbpositron')
folium.TileLayer('cartodbpositron').add_to(NY_map)

#apply the function we just created to every row in the dataframe
data_tree.apply(lambda x:  add_marker_tree(x), axis =1)
data_rats.apply(lambda x:  add_marker_rat(x), axis =1)
data_price.apply(lambda x:  add_marker_price(x), axis =1)
#data_places.apply(lambda x:  add_marker_place(x), axis =1)

#add borh marker groups to the map
#Places.add_to(NY_map)
Trees.add_to(NY_map)
Rats.add_to(NY_map)
BnB_prices.add_to(NY_map)

folium.map.LayerControl('topright', collapsed=False).add_to(NY_map)

#display map
NY_map

## Version 2

In [65]:
p_min = data_price.price.min(); p_max = data_price.price.max()
colormap = cm.LinearColormap(colors=['red','lightblue','blue'], 
                             index=[p_min,100,p_max],
                             vmin=p_min,vmax=p_max,
                             caption='Price'
                            )

In [66]:
#creating to different marker groups, one for each time period
BnB_prices = folium.FeatureGroup(name='<span style=\\"color: #0C0786;\\">Air BnBs (color after price) </span>')
Trees = folium.FeatureGroup(name='<span style=\\"color: #79D151;\\">Trees</span>')
Rats = folium.FeatureGroup(name='<span style=\\"color: #C0C0C0;\\">Rat Sightings</span>')
#Places = folium.FeatureGroup(name='<span style=\\"color: #5AD8CD;\\">Touristic Places</span>')

In [67]:
def add_marker_price(x):
      folium.CircleMarker(location=[x.latitude, x.longitude], radius= 2, color=colormap(x.price), weight = 2,  fill_color=colormap(x.price), fill=True, fill_opacity=1).add_to(BnB_prices)

def add_marker_tree(x):
      folium.CircleMarker(location=[x.latitude, x.longitude], radius= 1, color='#79D151', weight = 2,  fill_color='#79D151', fill=True, fill_opacity=1).add_to(Trees)

def add_marker_rat(x):
      folium.CircleMarker(location=[x.Latitude, x.Longitude], radius= 1, color='#C0C0C0', weight = 2,  fill_color='#C0C0C0', fill=True, fill_opacity=1).add_to(Rats)

def add_marker_place(x):
      folium.CircleMarker(location=[x.latitude, x.longitude], radius= 1, color='#5AD8CD', weight = 2,  fill_color='#5AD8CD', fill=True, fill_opacity=1).add_to(Places)

In [68]:
NYC_coor = (40.730610, -73.935242)

NY_map = folium.Map(NYC_coor, zoom_start=11, tiles = 'cartodbpositron')
folium.TileLayer('cartodbpositron').add_to(NY_map)

#apply the function we just created to every row in the dataframe
data_tree.apply(lambda x:  add_marker_tree(x), axis =1)
data_rats.apply(lambda x:  add_marker_rat(x), axis =1)
data_price.apply(lambda x:  add_marker_price(x), axis =1)
#data_places.apply(lambda x:  add_marker_place(x), axis =1)

#add borh marker groups to the map
BnB_prices.add_to(NY_map)
#Places.add_to(NY_map)
Trees.add_to(NY_map)
Rats.add_to(NY_map)

NY_map.add_child(colormap)
folium.map.LayerControl('topright', collapsed=False).add_to(NY_map)
NY_map.save('Map_features.html')
#display map
NY_map